<a href="https://colab.research.google.com/github/ImagingDataCommons/Cloud-Resources-Workflows/blob/notebooks2/Notebooks/Totalsegmentator/preProccessing_of_postProcessingExtractPerframe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**This notebook provides a step-by-step guide on how to generate a datatable for Terra. This datatable is essential for extracting the DICOM attribute, PerFrameFunctionalGroupsSequence, using the workflow linked below.**

You can find the PerFrameFunctionalGroupsSequence Extraction workflow [here](https://dockstore.org/workflows/github.com/ImagingDataCommons/Cloud-Resources-Workflows/perFrameFunctionalGroupSequenceExtractionOnTerra:main?tab=info).

The workflow requires manifests as inputs, each of which containining 10 (chosen arbitarily can be any number) batches of compressed DICOM SEGs, amounting to up to 120 DICOM SEG files. These manifests include URLs, accessible by a service account, that point to the compressed DICOM SEG files. These files are generated by the TotalSegmentator workflow on Terra.

Once these steps are completed, a datatable is produced and is ready to be uploaded to Terra's data tables, that can be referenced for  PerFrameFunctionalGroupsSequence Extraction workflow

###**Installing Packages (local)**

In [ ]:
%%capture
!sudo apt-get update \
  && apt-get install -y --no-install-recommends \
  lz4

In [ ]:
%%capture
!pip install pydicom \
   google-cloud-bigquery \
   pyarrow \
   db_dtypes

###**Importing Packages**

In [ ]:
from datetime import datetime
import os
import shutil
import pandas as pd
import pydicom
import traceback
import logging
from tqdm import tqdm
import subprocess

###**Example Terra datatable**

In [ ]:
segFilesCsv='https://raw.githubusercontent.com/ImagingDataCommons/Cloud-Resources-Workflows/notebooks/sampleManifests/sample_two_vm_workflow_datatable_on_terra.tsv'

###**Read the tsv from twoVMworkflow datatable on terra**

In [ ]:
data= pd.read_table(segFilesCsv)
data

,entity:twoVM_060823_id,dcm2niixErrors,dicomsegAndRadiomicsSR_CompressedFiles,dicomsegAndRadiomicsSR_Errors,dicomsegAndRadiomicsSR_OutputNotebook,dicomsegAndRadiomicsSR_UsageMetrics,dicomToNiftiConverterTool,downloadDicomAndConvertAndInferenceTotalSegmentatorOutputNotebook,downloadDicomAndConvertAndInferenceTotalSegmentatorUsageMetrics,pyradiomicsRadiomicsFeatures,s5cmdUrls,structuredReportsDICOM,structuredReportsJSON,totalsegmentatorErrors
0,batch1,NaN,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,NaN,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,dcm2niix,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/n...,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,NaN
1,batch10,NaN,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,NaN,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,dcm2niix,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/n...,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,NaN
2,batch11,NaN,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,NaN,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,dcm2niix,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/n...,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,NaN
3,batch12,NaN,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,NaN,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,dcm2niix,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/n...,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,NaN
4,batch13,NaN,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,NaN,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,dcm2niix,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/n...,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,batch84,NaN,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,NaN,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,dcm2niix,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/n...,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,NaN
83,batch85,NaN,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,NaN,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,dcm2niix,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/n...,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,NaN
84,batch86,NaN,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,NaN,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,gs://fc-

###**Copy the files from Terra bucket to another bucket where you can create service accounts**

In [ ]:
# Add a new column called 'new_destination'
project_id='test_project'
bucketname='test_bucket'
first_column = data.columns[0]  # Get the label of the first column
# Populate the 'new_destination' column with values that are a concatenation of 'bucket_id' and 'batch_id'
data['new_destination'] = data["dicomsegAndRadiomicsSR_CompressedFiles"] +' '+f'gs://{bucketname}/' + data[first_column].astype(str)+'/'
data

,entity:twoVM_060823_id,dcm2niixErrors,dicomsegAndRadiomicsSR_CompressedFiles,dicomsegAndRadiomicsSR_Errors,dicomsegAndRadiomicsSR_OutputNotebook,dicomsegAndRadiomicsSR_UsageMetrics,dicomToNiftiConverterTool,downloadDicomAndConvertAndInferenceTotalSegmentatorOutputNotebook,downloadDicomAndConvertAndInferenceTotalSegmentatorUsageMetrics,pyradiomicsRadiomicsFeatures,s5cmdUrls,structuredReportsDICOM,structuredReportsJSON,totalsegmentatorErrors,new_destination
0,batch1,NaN,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,NaN,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,dcm2niix,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/n...,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,NaN,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...
1,batch10,NaN,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,NaN,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,dcm2niix,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/n...,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,NaN,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...
2,batch11,NaN,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,NaN,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,dcm2niix,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/n...,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,NaN,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...
3,batch12,NaN,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,NaN,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,dcm2niix,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/n...,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,NaN,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...
4,batch13,NaN,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,NaN,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,dcm2niix,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/n...,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,NaN,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,batch84,NaN,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,NaN,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,dcm2niix,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/n...,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,NaN,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...
83,batch85,NaN,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,NaN,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,dcm2niix,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/s...,gs://fc-5af

In [ ]:
!gcloud auth login

In [ ]:
!gcloud config set project $project_id

In [ ]:
urls= data['new_destination'].to_list()

In [ ]:
for url in urls:
  !gsutil cp {url}

###**Generate manifests for Terra datatable**

In [ ]:
import os
import pandas as pd

data["batch_number"] = data[data.columns[0]].str.extract("(\d+)").astype(int)
data = data.sort_values("batch_number")
data = data.drop(columns=["batch_number"])

# Create 'urls' directory if it doesn't exist
if not os.path.exists("urls"):
    os.makedirs("urls")

# Split DataFrame into batches of 10 rows each
num_batches = len(data) // 10 + (len(data) % 10 > 0)

for i in range(num_batches):
    batch = data.iloc[i * 10 : (i + 1) * 10]
    batch.to_csv(f"urls/batch_{i+1}.csv", index=False)


###**Upload manifests to Terra bucket and Generate Terra datatable**

In [ ]:
now = datetime.now().strftime("%Y_%m_%d_%H_%M")
terra_bucket = "test-terra-bucket"
folder_in_terra_bucket = "perframe"

entity_column_name = f"entity:perframe_{now}_id"

# Initialize a list to store each row as a DataFrame
rows = []

# Iterate over each file in the 'urls' directory
for filename in os.listdir("urls"):
    if filename.endswith(".csv"):
        # Extract the batch number from the filename
        entity = filename.split("_")[1].split(".")[0]

        # Construct the GCS URL for this batch's file
        gcsurl = os.path.join(terra_bucket, folder_in_terra_bucket, filename)

        !gsutil cp filename gcsurl

        # Append a new row to the list as a DataFrame
        rows.append(pd.DataFrame({entity_column_name: [entity], "gcsurl": [gcsurl]}))

# Concatenate all the DataFrames in the list
terra_df = pd.concat(rows, ignore_index=True)

# Save the DataFrame to a .tsv file
terra_df = terra_df.sort_values(by=entity_column_name)
terra_df.to_csv("terra_datatable.tsv", sep="\t", index=False)
terra_df


,entity:perframe_2023_12_15_02_09_id,gcsurl
6,1,test-terra-bucket/perframe/batch_1.csv
8,2,test-terra-bucket/perframe/batch_2.csv
2,3,test-terra-bucket/perframe/batch_3.csv
4,4,test-terra-bucket/perframe/batch_4.csv
1,5,test-terra-bucket/perframe/batch_5.csv
0,6,test-terra-bucket/perframe/batch_6.csv
3,7,test-terra-bucket/perframe/batch_7.csv
7,8,test-terra-bucket/perframe/batch_8.csv
5,9,test-terra-bucket/perframe/batch_9.csv
